In [1]:
# HYBRID OF TEXT EXTRACTION TOOLS WITH TESSERACT OCR. CONVERT PDF TO MARKDOWN FORMAT.

import os
import fitz
from pdf2image import convert_from_path
import pytesseract
import pypandoc


def extract_text_or_perform_ocr(pdf_path, page_number):
    """Extracts text from a PDF page; performs OCR if the page is image-based."""
    doc = fitz.open(pdf_path)
    page = doc[page_number]
    text = page.get_text().strip()
    doc.close()

    if not text:  # If no text, perform OCR on the page
        images = convert_from_path(pdf_path, first_page=page_number + 1, last_page=page_number + 1)
        text = pytesseract.image_to_string(images[0])
    
    return text

def convert_pdf_to_md(pdf_path, output_path):
    """Converts a PDF file to Markdown format using Pandoc."""
    doc = fitz.open(pdf_path)
    # Extract text from each page (with OCR for image-based pages)
    texts = [extract_text_or_perform_ocr(pdf_path, pn) for pn in range(len(doc))]
    doc.close()

    full_text = '\n\n'.join(texts)

    # Writing the full extracted text to a temporary .txt file
    temp_txt_path = "temp_extracted_text.txt"
    with open(temp_txt_path, 'w', encoding='utf-8') as temp_file:
        temp_file.write(full_text)

    # Convert the text file to Markdown using Pandoc
    output_md = pypandoc.convert_file(temp_txt_path, 'md', format='markdown', outputfile=output_path)

    os.remove(temp_txt_path)  # Clean up the temporary file

    return output_md

def batch_convert_pdf_to_md(pdf_folder, output_folder):
    """Converts all PDF files in a folder to Markdown format."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            md_filename = os.path.splitext(pdf_file)[0] + '.txt'
            output_path = os.path.join(output_folder, md_filename)
            convert_pdf_to_md(pdf_path, output_path)
            print(f"Converted '{pdf_file}' to Markdown format as '{md_filename}'.")

# Example usage
pdf_folder = 'C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\AF_pdf'
output_folder = 'C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\af_txt'
batch_convert_pdf_to_md(pdf_folder, output_folder)


Converted '12038_12038_2020-05-08-SierraLeone_AF-Inception Workshop Report.pdf' to Markdown format as '12038_12038_2020-05-08-SierraLeone_AF-Inception Workshop Report.txt'.
Converted '12038_12038_Project Proposal Sierra Leone Adaptation Fund IFAD_13May2019CLEAN.pdf' to Markdown format as '12038_12038_Project Proposal Sierra Leone Adaptation Fund IFAD_13May2019CLEAN.txt'.
Converted '12039_12039_Clean for posting .pdf' to Markdown format as '12039_12039_Clean for posting .txt'.
Converted '12039_12039_Inception report_AF project_Tajikistan.pdf' to Markdown format as '12039_12039_Inception report_AF project_Tajikistan.txt'.
Converted '12040_12040_PROJECT LAUNCH AND INCEPTION REPORT FOR SAINT LUCIA AF PROJECT.pdf' to Markdown format as '12040_12040_PROJECT LAUNCH AND INCEPTION REPORT FOR SAINT LUCIA AF PROJECT.txt'.
Converted '12040_12040_St Lucia for posting.pdf' to Markdown format as '12040_12040_St Lucia for posting.txt'.
Converted '12093_12093_Congo Clean.pdf' to Markdown format as '120

In [ ]:
# ALTEWRNATIVE EXTRACTION TOOLS WITH EASYOCR. CONVERT PDF TO MARKDOWN FORMAT.

import os
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import easyocr
import pypandoc
import numpy as np  # Add this import at the beginning of your script

def extract_text_or_perform_ocr(pdf_path, page_number):
    """Extracts text from a PDF page; performs OCR if the page is image-based."""
    doc = fitz.open(pdf_path)
    page = doc[page_number]
    text = page.get_text().strip()
    doc.close()

    if not text:  # If no text, perform OCR on the page
        images = convert_from_path(pdf_path, first_page=page_number + 1, last_page=page_number + 1)
        reader = easyocr.Reader(['en'])  # Initialize EasyOCR reader, specify language as needed
        image_np = np.array(images[0])  # Convert PIL image to NumPy array
        results = reader.readtext(image_np, paragraph=True)  # Use `paragraph` for better grouping
        text = ' '.join([result[1] for result in results])
    
    return text

def convert_pdf_to_md(pdf_path, output_path):
    """Converts a PDF file to Markdown format using Pandoc."""
    doc = fitz.open(pdf_path)
    texts = [extract_text_or_perform_ocr(pdf_path, pn) for pn in range(len(doc))]
    doc.close()

    full_text = '\n\n'.join(texts)
    temp_txt_path = "temp_extracted_text.txt"
    with open(temp_txt_path, 'w', encoding='utf-8') as temp_file:
        temp_file.write(full_text)

    output_md = pypandoc.convert_file(temp_txt_path, 'md', format='markdown', outputfile=output_path)
    os.remove(temp_txt_path)  # Clean up the temporary file

    return output_md

def batch_convert_pdf_to_md(pdf_folder, output_folder):
    """Converts all PDF files in a folder to Markdown format."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            md_filename = os.path.splitext(pdf_file)[0] + '.txt'
            output_path = os.path.join(output_folder, md_filename)
            convert_pdf_to_md(pdf_path, output_path)
            print(f"Converted '{pdf_file}' to Markdown format as '{md_filename}'.")

# Example usage
pdf_folder = 'pdf'
output_folder = 'txt'
batch_convert_pdf_to_md(pdf_folder, output_folder)


In [ ]:
import os
import re

def process_text(text):
    text = re.sub(r"([a-zA-Z])(\n)([A-Z])", r"\1. \3", text)  # Adds missing dot between sentences split across lines.
    text = re.sub(r"([a-zA-Z])\n", r"\1 ", text)  # Removes inappropriate line breaks within paragraphs.
    #text = re.sub(r"\s{2,}", " ", text)  # Replace multiple spaces with a single space, if needed.
    return text

def process_files(directory_path):
    if not os.path.exists(directory_path):
        print("Directory does not exist:", directory_path)
        return
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory_path, filename)
            try:
                with open(filepath, 'r', encoding='utf-8') as file:
                    content = file.read()

                processed_content = process_text(content)

                # Change the file extension from .md to .txt
                txt_filename = os.path.splitext(filename)[0] + '.txt'
                txt_filepath = os.path.join(directory_path, txt_filename)

                with open(txt_filepath, 'w', encoding='utf-8') as file:
                    file.write(processed_content)
                print(f"Processed and saved {txt_filename}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

directory_path = 'txt'
process_files(directory_path)


# Work for .docx, .docm, and .dotx 

TODO:// .doc, .dot Check with Cesare if is not faster to do manual conversion

In [6]:
import os
from docx import Document
import glob

def process_document(doc_path):
    print(f"Processing {doc_path}...")
    doc = Document(doc_path)
    output = []

    for paragraph in doc.paragraphs:
        output.append(paragraph.text + "\n")

    for table in doc.tables:
        output.append("\n[Table Start]\n")
        for row in table.rows:
            row_data = []
            for cell in row.cells:
                cell_text = cell.text.replace('\n', ' ').strip()
                row_data.append(cell_text)
            output.append(" | ".join(row_data) + "\n")
        output.append("[Table End]\n")
    
    return ''.join(output)

def save_text(output_text, output_path):
    with open(output_path, 'w', encoding='utf-8') as file:
        file.write(output_text)

def process_folder(folder_path, output_folder):
    for doc_path in glob.glob(os.path.join(folder_path, '*.docx')):
        structured_text = process_document(doc_path)
        
        # Generate output path based on the document name
        output_file_name = os.path.basename(doc_path).replace('.docx', '.txt')
        output_path = os.path.join(output_folder, output_file_name)
        
        save_text(structured_text, output_path)
        print(f"Saved processed text to {output_path}")

# Path to your folder containing Word documents
folder_path = "C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\af_word_docs"

# Desired output folder for the text files
output_folder ='C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\af_word_text'

# Process all documents in the folder
process_folder(folder_path, output_folder)


Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_word_docs\10_10_Madagascar Adaptation Fund Project_MTR report AFrice Final_clean.docx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_word_text\10_10_Madagascar Adaptation Fund Project_MTR report AFrice Final_clean.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_word_docs\12094_DiMMA Gender & Youth  Strategy 2021.docx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_word_text\12094_DiMMA Gender & Youth  Strategy 2021.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_word_docs\12100_INCEPTION REPORT ENGLISH-HAI.docx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_word_text\12100_INCEPTION REPORT ENGLISH-HAI.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_word_docs\1430_1430_3 - AF Preconcept ACREI 8th February Clean 4 FEB.docx...
Saved processed text to

# Excel for .xls 


In [3]:
import os
import glob
import pandas as pd
import xlrd 

def excel_to_txt(excel_path, txt_path):
    with pd.ExcelFile(excel_path) as xls:
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            for sheet_name in xls.sheet_names:
                txt_file.write(f"--- {sheet_name} ---\n")
                df = pd.read_excel(xls, sheet_name=sheet_name)
                for index, row in df.iterrows():
                    # Process each cell, append a period if the cell is not empty.
                    row_values = [(str(cell) + '.' if pd.notnull(cell) and str(cell).strip() != '' else '') for cell in row]
                    txt_file.write('\t'.join(row_values) + '\n')
                txt_file.write('\n')  # Separate sheet with a space
                
def process_folder(source_folder, target_folder):
    # Search for both .xls and .xlsx files
    for excel_file in glob.glob(os.path.join(source_folder, '*.xls*')):
        file_name = os.path.basename(excel_file)
        # Preserve original extension in replacement to handle both .xls and .xlsx
        txt_file_name = file_name.rsplit('.', 1)[0] + '.txt'
        txt_path = os.path.join(target_folder, txt_file_name)
        
        print(f"Processing {excel_file}...")
        excel_to_txt(excel_file, txt_path)
        print(f"Saved processed text to {txt_path}")

# Update these paths to your source and target folders
source_folder = r"C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel"
target_folder = r"C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text"

process_folder(source_folder, target_folder)


Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\10_10_ATTRZVIH - 1st PPR for website.xls...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\10_10_ATTRZVIH - 1st PPR for website.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\10_10_For Website_AF RICE PPR - final for AF submission - revised 270117.xls...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\10_10_For Website_AF RICE PPR - final for AF submission - revised 270117.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\10_10_For Website_Madagascar PPRA Oct 2013-oCT 2014 REVISED-Oct  2015. FINAL for AF.xls...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\10_10_For Website_Madagascar PPRA Oct 2013-oCT 2014 REVISED-Oct  2015. FINAL for AF.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\10_10_PPR Mada

c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will b

Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\12_12_For web PPR_PACV_AF_2019 060120_Final.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\12_12_For Website_4789_AF_Mali_PPR 2_revised June 2018.xlsx...


c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will b

Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\12_12_For Website_4789_AF_Mali_PPR 2_revised June 2018.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\12_12_For Website_Revised PPR _ Mali AF Project PIMS 4789_18 Oct 2017.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\12_12_For Website_Revised PPR _ Mali AF Project PIMS 4789_18 Oct 2017.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\12_12_web_Final PPR PIMS_4789_du 9 mars 2021.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\12_12_web_Final PPR PIMS_4789_du 9 mars 2021.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\1417_1417_PPR II-Ayninacuy_Project_for_web.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\1417_1417_PPR II-Ayninacuy_Project_for_web.txt
Proces

...

Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\69_69_For website_ Project Performance Report 2_22102013_revised.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\69_69_For Website_PPR 3_Template 2015_9 agosto.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\69_69_For Website_PPR 3_Template 2015_9 agosto.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\69_69_For Website_PPR 4_Template 2016 - 2.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\69_69_For Website_PPR 4_Template 2016 - 2.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\69_69_For website_Uruguay PPR 1 final .xlsx...


c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will b

Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\69_69_For website_Uruguay PPR 1 final .txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\69_69_web_Copy of PPRTemplate 2017_26 march 2019.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\69_69_web_Copy of PPRTemplate 2017_26 march 2019.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\69_69_web_PPR Template 2018.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\69_69_web_PPR Template 2018.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\69_69_web_PPRTemplate 2019.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\69_69_web_PPRTemplate 2019.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\6_6_CLEARED FOR WEB_Copy of 4952_AF_Ghana_PPR_Aug 2019_re

c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will b

Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\9969_30012023PPR_Year 1_SWAHAT revised 22 Feb 2023 (Rev4) - for web.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\9_9_ADA-PPR_2019_For_Web.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\9_9_ADA-PPR_2019_For_Web.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\9_9_For Website_170412 PPR_PACCZO-2016.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\9_9_For Website_170412 PPR_PACCZO-2016.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\9_9_web 280119_ADA-Morocco_PPR FINAL.xlsx...
Saved processed text to C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel_text\9_9_web 280119_ADA-Morocco_PPR FINAL.txt
Processing C:\Users\david\My Drive\data\analysis_git_data\cgiar\af_excel\9_9_web_ADA-PPR_2020_V reviwed_cle

Save different headings in different columns  WORD

//TODO: Check with Cesare if the following approaches can work. Still a bunch of cleaning but could also be a way to split sentence.

In [2]:
import pandas as pd
def process_document_to_df(doc_path):
    doc = Document(doc_path)
    data = {}
    current_heading = None

    for paragraph in doc.paragraphs:
        if paragraph.style.name.startswith('Heading'):
            current_heading = paragraph.text
            data[current_heading] = []
        else:
            if current_heading:
                data[current_heading].append(paragraph.text)
            else:
                pass
    df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()]))
    return df


# Path to your Word document
doc_path = "C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\AF_docs\\50_50_MTR-AF-Uzbekistan-Report-FINAL2.docx"

# Desired output path for the text file
output_path ='C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar'

# Process the document and get structured text
structured_text = process_document_to_df(doc_path)




NameError: name 'Document' is not defined

TRY TO SAVE ONLY FROM H1 TO H1

In [ ]:

def process_document_to_df(doc_path):
    doc = Document(doc_path)
    data = {}
    current_heading = None
    current_content = []

    for paragraph in doc.paragraphs:
        if paragraph.style.name == 'Heading 1':
            if current_heading is not None:
                data[current_heading] = "\n".join(current_content)
                current_content = []
            current_heading = paragraph.text
            data[current_heading] = []
        else:
            if current_heading:
                current_content.append(paragraph.text)

    if current_heading is not None and current_heading not in data:
        data[current_heading] = "\n".join(current_content)

    df = pd.DataFrame(list(data.items()), columns=['Heading', 'Content'])
    return df

# Directory containing your Word documents
doc_directory = "C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\sample_word"

dfs = []

for filename in os.listdir(doc_directory):
    if filename.endswith(".docx"):
        doc_path = os.path.join(doc_directory, filename)
        
        print(f"Processing document: {filename}")  # Print statement added here
        df = process_document_to_df(doc_path)
        
        df['Document'] = filename
        dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True, sort=False)

# Optional: save the combined DataFrame to a CSV file
output_csv_path = os.path.join(doc_directory, 'combined_documents.csv')
combined_df.to_csv(output_csv_path, index=False)

print(f"Combined DataFrame saved to {output_csv_path}")